In [2]:
#!pip install -U qiskit
#!pip install -U qiskit_nature
#!pip install -U qiskit_aer
#!pip install -U qiskit_algorithms
#!pip install -U qiskit_ibm_runtime
#!pip install -U qiskit-aer-gpu
#!pip freeze | grep qiskit

In [1]:
### Qiskit Nature
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal


from qiskit_nature.second_q.mappers import BravyiKitaevMapper,BravyiKitaevSuperFastMapper,JordanWignerMapper,ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC, UCCSD
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, ExcitedStatesEigensolver, QEOM, EvaluationRule
from qiskit_nature.second_q.transformers import FreezeCoreTransformer


### Qiskit Runtime
from qiskit.primitives import BackendEstimator as Estimator
from qiskit import QuantumCircuit,transpile
from qiskit_aer import AerSimulator, Aer
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService


### Qiskit Algorithms
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, SLSQP,COBYLA,SPSA
from qiskit_algorithms import NumPyMinimumEigensolver


from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint

### Other
import numpy as np
import copy
import warnings
warnings.filterwarnings("ignore")
from decimal import Decimal

In [2]:
# Settings
###################################
basis = 'sto-3g'
unit = DistanceUnit.ANGSTROM
qubit_reduction = False
map_type = "JordanWigner"
shots = 1000
#optimizer = SLSQP(maxiter=10,ftol=1e-9))
optimizer = SPSA(maxiter=10)
ini_params = [ 0, 0, 0]
dx = 1e-5  # displacement for numerical gradient
###################################

In [3]:
class Molecule:
    def __init__(self):
        self.atoms = []
        self.energy = None
        self.gradient = None

    def add_atom(self, atom_type: str, x: float, y: float, z: float):
        atom = {
            "type": atom_type,
            "coordinates": (x, y, z)
        }
        self.atoms.append(atom)

    def load_from_file(self, filename: str):
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Reading the atoms section
        for line in lines[3:]:  # Skip the first three lines
            if line.strip() == "*":  # End
                break
            parts = line.split()
            atom_type = parts[0]
            x, y, z = map(float, parts[1:4])
            self.add_atom(atom_type, x, y, z)

    def get_pyscf_driver(self, unit: DistanceUnit = DistanceUnit.ANGSTROM, basis: str = "sto-3g"):
        # Create molecular geometry string
        geometry_str = "\n".join(f"{atom['type']} {atom['coordinates'][0]} {atom['coordinates'][1]} {atom['coordinates'][2]}"
                                 for atom in self.atoms)
        
        # Generate and return the PySCFDriver
        driver = PySCFDriver(atom=geometry_str, unit=unit, basis=basis)
        
        problem = driver.run()
        
        return problem
    
    def displace_atom(self, index: int, dx: tuple):
        if index < 0 or index >= len(self.atoms):
            raise IndexError("Atom index is out of range.")
    
        # Convert to Decimal for high-precision arithmetic
        self.atoms[index]["coordinates"] = [
            float(Decimal(str(self.atoms[index]["coordinates"][i])) + Decimal(str(dx[i]))) for i in range(3)]
    
    
    def displace_atom_coordinate(self, atom_index: int, axis: int, dx: float):
        if atom_index < 0 or atom_index >= len(self.atoms):
            raise IndexError("Atom index is out of range.")
        if axis not in (0, 1, 2):
            raise ValueError("Axis must be 0 (x), 1 (y), or 2 (z).")

        # Update the specified coordinate - change
        self.atoms[atom_index]["coordinates"][axis] += dx
    
    def print_atoms(self):
        if not self.atoms:
            print("No atoms in the molecule.")
        else:
            print("Atoms in the molecule:")
            for i, atom in enumerate(self.atoms):
                print(f"{i + 1}. {atom['type']} - Coordinates: {atom['coordinates']}")
        
    def __repr__(self):
        atom_repr = "\n".join(f"{atom['type']}: {atom['coordinates']}" for atom in self.atoms)
        return f"Energy: {self.energy}\nGradient: {self.gradient}\nAtoms:\n{atom_repr}"

    

In [4]:
def compute_energy_and_grad(mol,dx, shots, qubit_reduction, mapper_type, parameters=None, opt=SLSQP()):


    # Molecule displacement setup
    #############################################################################
    
    # r
    molecule = mol
    problem = molecule.get_pyscf_driver(unit=unit,basis=basis)
    e_nr = problem.nuclear_repulsion_energy
    main_op = problem.hamiltonian.second_q_op()
    
    molecule.print_atoms()
    
    # r - dx
    molecule_m = copy.deepcopy(mol)
    molecule_m.displace_atom(1, (0, 0, -dx))
    molecule_m.print_atoms()
    #molecule_m.displace_atom_coordinate(1, 2, -dx)
    problem_mdx = molecule_m.get_pyscf_driver(unit=unit,basis=basis)
    e_nr_mdx = problem_mdx.nuclear_repulsion_energy
    second_q_ops_mdx = problem_mdx.second_q_ops()
    main_op_mdx = problem_mdx.hamiltonian.second_q_op()

    # r + dx
    molecule_p = copy.deepcopy(mol)
    molecule_p.displace_atom(1, (0, 0, dx))
    molecule_p.print_atoms()
    #molecule_p.displace_atom_coordinate(1, 2, dx)
    problem_pdx = molecule_p.get_pyscf_driver(unit=unit,basis=basis)
    e_nr_pdx = problem_pdx.nuclear_repulsion_energy
    second_q_ops_pdx = problem_pdx.second_q_ops()
    main_op_pdx = problem_pdx.hamiltonian.second_q_op()
    
    
    
    #############################################################################

    num_particles = (problem.num_alpha,
                    problem.num_beta)

    num_spatial_orbitals = problem.num_spatial_orbitals
    num_spin_orbitals = 2 * problem.num_spatial_orbitals
    
    
    print(f'Nuclear repulsion energy: {problem.nuclear_repulsion_energy} Ha')
    print(f'Reference energy: {problem.reference_energy} Ha')
    print(f'Number of spin orbitals: {problem.num_spin_orbitals}')
    print(f'Number of alpha electrons: {problem.num_alpha}')
    print(f'Number of beta electrons: {problem.num_beta}')
    
    #############################################################################
    
    
    # MAPPERS
    #############################################################################
    mapper = None
    if mapper_type == 'JordanWigner':
        mapper = JordanWignerMapper()
    elif mapper_type == 'ParityMapper':
        if qubit_reduction == False:
            mapper = ParityMapper(num_particles=problem.num_particles)
        else:
            mapper = ParityMapper(num_particles=problem.num_particles,two_qubit_reduction=True)
    elif mapper_type == 'BravyiKitaev':
        mapper = BravyiKitaevMapper()
    elif mappper_type == 'Superfast':
        mapper = BravyiKitaevSuperFastMapper(num_particles=problem.num_particles)

    # JordanWigner - https://web.archive.org/web/20191103083720/http://michaelnielsen.org/blog/archive/notes/fermions_and_jordan_wigner.pdf
    # Parity - https://arxiv.org/pdf/1701.08213
    # BravyiKitaev - https://www.sciencedirect.com/science/article/abs/pii/S0003491602962548
    # Superfast - https://arxiv.org/pdf/1712.00446

    # map to qubit operators
    qubit_op = mapper.map(main_op)
    qubit_op_mdx = mapper.map(main_op_mdx)
    qubit_op_pdx = mapper.map(main_op_pdx)

    #############################################################################

    # Classical - Exact solution with NUMPY
    #############################################################################
    numpy_solver = NumPyMinimumEigensolver()
    ret_exact = numpy_solver.compute_minimum_eigenvalue(qubit_op)
    ret_exact_mdx = numpy_solver.compute_minimum_eigenvalue(qubit_op_mdx)
    ret_exact_pdx = numpy_solver.compute_minimum_eigenvalue(qubit_op_pdx)
    e_fci = ret_exact._eigenvalue.real + e_nr
    e_fci_mdx = ret_exact_mdx.eigenvalue.real + e_nr_mdx
    e_fci_pdx = ret_exact_pdx.eigenvalue.real + e_nr_pdx
    grad_fci = (e_fci_pdx - e_fci_mdx) / 2 / dx
    #############################################################################

    # PREPARE QUANTUM CIRCUIT
    #############################################################################
    
    # BACKEND
    #############################################################################
    aer_sim = AerSimulator(method="matrix_product_state")
    #aer_sim = AerSimulator(method="statevector")
    #aer_sim = AerSimulator(method="statevector",device='GPU')
    estimator = Estimator(backend=aer_sim)

    
    # SETUP ANSATZ
    #############################################################################
    
    # QURI functionality - maybe overwrite
    # https://docs.quantum.ibm.com/guides/qunasys-quri-chemistry#quri-chemistry-with-uccsd
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    
    #ansatz = UCC(num_spatial_orbitals=num_spatial_orbitals, num_particles=num_particles, qubit_mapper=mapper,
    #           excitations='d')

    vqe_solver = VQE(estimator, ansatz=ansatz, optimizer=opt)
    
    # SETUP INITIAL POINT
    #############################################################################
    
    
    vqe_solver.initial_point = np.zeros(ansatz.num_parameters)
    
    
    initial_point = HFInitialPoint()
    initial_point.ansatz = ansatz
    initial_point.problem = problem
    vqe_solver.initial_point = initial_point.to_numpy_array()
    
    
    # COMPUTE - QUANTUM
    #############################################################################

    solver = GroundStateEigensolver(mapper, vqe_solver)
    
    result_cs = []
    if solver.supports_aux_operators():
        result_cs = solver.solve(problem, aux_operators={'qubit_op_mdx': qubit_op_mdx, 'qubit_op_pdx': qubit_op_pdx})
    print(result_cs)

    
    
    #############################################################################
    e = result_cs.eigenvalues + e_nr

    e_pdx = float(result_cs.aux_operators_evaluated[0].get('qubit_op_pdx')) + e_nr_pdx
    e_mdx = float(result_cs.aux_operators_evaluated[0].get('qubit_op_mdx')) + e_nr_mdx
    
    e_pdx = Decimal(str(e_pdx))
    e_mdx = Decimal(str(e_mdx))
    dx = Decimal(str(dx))
    
    grad_cs = ((e_pdx - e_mdx) / (2*dx))
    
    #############################################################################
    print("\n\n")
    print("Nuclear repulsion energy:", e_nr)
    print("Electronic groundstate energy:", result_cs.groundenergy)
    print("Overall Energy: ", e)
    print("Energy in +:", e_pdx)
    print("Energy in -:", e_mdx)
    print("Gradient (CS):", grad_cs)
    print("\n\n")
    #print("Gradient (brute):", grad_brute)
    print("Exact energy:", e_fci)
    print("Exact energy in +:", e_fci_pdx)
    print("Exact energy in -:", e_fci_mdx)
    print("Gradient (FCI):", grad_fci)

    
    # Energy, energy + dx, energy - dx, gradient correlated_sampling # parameters of VQE
    return e, e_pdx, e_mdx, grad_cs, result_cs


In [5]:
molecule = Molecule()
molecule.load_from_file("example2.txt")
print(molecule)

Energy: None
Gradient: None
Atoms:
H: (0.0, 0.0, 0.0)
H: (0.0, 0.0, 0.735)


In [6]:
energy, energy_p, energy_m, gradient, parameters = compute_energy_and_grad(molecule ,dx,\
                                                                           shots, qubit_reduction, \
                                                                           map_type, parameters=None, \
                                                                           opt=optimizer)



####output to QCxMS


Atoms in the molecule:
1. H - Coordinates: (0.0, 0.0, 0.0)
2. H - Coordinates: (0.0, 0.0, 0.735)
Atoms in the molecule:
1. H - Coordinates: (0.0, 0.0, 0.0)
2. H - Coordinates: [0.0, 0.0, 0.73499]
Atoms in the molecule:
1. H - Coordinates: (0.0, 0.0, 0.0)
2. H - Coordinates: [0.0, 0.0, 0.73501]
Nuclear repulsion energy: 0.7199689944489797 Ha
Reference energy: -1.116998996754004 Ha
Number of spin orbitals: 4
Number of alpha electrons: 1
Number of beta electrons: 1
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.165219593411
  - computed part:      -1.165219593411
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -0.445250598962
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.309 S^2: 0.405 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  2.633006665364]
    - computed part:      [0.0  0.0  2.633006665364]
  